<h3 align="center">Introduction /  Business problem</h3>    

The consulting company I work for has received a new challenging request from a client, a multinational corporation: expanding the corporation’s business activities in Toronto, Canada. As a result, I have been assigned the task of identifying the most suitable location of the city where to establish the corporation’s new headquarters.

The client has one main concern: finding a location that can be livable for members of the international community.

We would identify areas with most promising characteristics and their advantages will then be clearly expressed, so that best possible final location may be chosen by our stakeholders.

The final assessment is based on the ability of Toronto to attract foreigners and look appealing to members of the international community. The most common venues of the city are identified with the Foursquare API. The k-means algorithm is then used to cluster venue categories. A final assessment of quantitative (number of venues) and qualitative (venue categories) variables is performed to determine the city that best suits the needs of the corporation. 

<h3 align="center">Description of the data</h3> 

1. The data to create the dataframe is web-scrapped from Wikipedia. The data is then wrangled and merged with another file including coordinates (latitude and longitude).

2. Additional data is taken from Foursquare in order to identify the most attractive venues for the corporation.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# Import packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

# Download data and extract the table with Beautiful Soup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
# Extract columns and rows from the Wikipedia table
A=[]
B=[]
C=[]
 
for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Create the dataframe with pandas

In [3]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop boroughs with the value 'Not assigned'.

In [4]:
df = df[df.Borough != 'Not assigned']

Import numpy, merge rows based on same Postal Code area and reset the index.

In [5]:
# Import Numpy
import numpy as np
# Merge rows based on same Postal Code area
df.groupby('Postal Code').sum().replace(0,np.nan)
# Reset index
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Change of 'Not assigned' neighborhood values to borough's values.

In [6]:
df.groupby(['Postal Code'])['Neighborhood'].apply(','.join).reset_index()

,Postal Code,Neighborhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood\n,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae\n
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park\n,Ionview,Kennedy Park"
7,M1L,"Clairlea,Golden Mile,Oakridge"
8,M1M,"Cliffcrest,Cliffside,Scarborough Village West\n"
9,M1N,"Birch Cliff,Cliffside West\n"


Upload latitude and longitude values using a .csv file and merge them with the other dataframe.

In [7]:
df2 = pd.read_csv(r'C:\Users\Dade\Downloads\Geospatial_Coordinates.csv')
# Merge databases
df = pd.merge(df, df2)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Convert an address into latitude and longitude values.

In [8]:
from geopy.geocoders import Nominatim 
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Creating a map of Toronto with neighborhoods superimposed on top.

In [9]:
# Import Folium and create toronto map

import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

Now I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [10]:
CLIENT_ID = 'RRSD5LFYQIYSOIRFQVBW2NOBJGIECUSRFGYEHBVXDDTPA1GM' # your Foursquare ID
CLIENT_SECRET = 'Y2T531BVTD30X5TVAZ4OJ25CQYXW2YDB4ZLA4KSKISVNIA2Z' # your Foursquare Secret
VERSION = '20190328' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RRSD5LFYQIYSOIRFQVBW2NOBJGIECUSRFGYEHBVXDDTPA1GM
CLIENT_SECRET:Y2T531BVTD30X5TVAZ4OJ25CQYXW2YDB4ZLA4KSKISVNIA2Z


Explore Neighborhoods in Toronto.

Let's borrow the 'get_category_type' function from the Foursquare lab.

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to retrieve nearby venues in Toronto. I will use 1000 meters as radius.

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods


KeyError: 'groups'

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

How many venues were returned for each neighborhood?

In [ ]:
toronto_venues.groupby('Neighborhood').count()

How many unique categories can be extracted from all the returned venues?

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Analyze each neighborhood.

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

How big is the new dataframe?

In [ ]:
toronto_onehot.shape

Now let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

We will also print each neighborhood along with the top 5 most common venues.

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Now we will create a new dataframe with the new values and display the top 10 venues for each neighborhood.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

We will now move on and cluster the neighborhood using the 'k-means' machine learning technique. Then ,we will add the clusters to the dataframe showing the top venues in Toronto's neighborhoods.

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df.head() # check the last columns!

We will now visualize the resulting clusters.

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]